# View the 5D beam density in normalized slit-screen coordinates

$x_1$ = position of first vertical slit \
$x_2$ = position of second horizontal slit \
$y_1$ = position of horizontal slit \
$y_3$ = y position at view screen \
$x_3$ = x position at view screen

In [ ]:
import sys
import os
from os.path import join
import time
from datetime import datetime
import importlib
import numpy as np
import pandas as pd
import h5py
import imageio
from scipy import ndimage
from scipy import interpolate
import skimage
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from matplotlib import pyplot as plt
from matplotlib import colors
import plotly.graph_objs as go
from ipywidgets import interact
import proplot as pplt

sys.path.append('../..')
from tools import energyVS06 as energy
from tools import image_processing as ip
from tools import plotting as mplt
from tools import utils
from tools.utils import project

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

## Load data

In [ ]:
folder = '_output'
filenames = os.listdir(folder)
for filename in filenames:
    if filename.startswith('coordinates3d_raw') or filename.startswith('f_raw'):
        print(filename)

In [ ]:
filename = 'f_raw_merged2-211203151919-211203192701-scan-xxpy-image-ypdE.mmp'
coordfilename = 'coordinates3d_raw_merged2-211203151919-211203192701-scan-xxpy-image-ypdE.npy'

In [ ]:
info = utils.load_pickle('_output/info.pkl')
info

In [ ]:
shape = info['rawgrid_shape']  # (x1, x2, y1, y3, x3)
dtype = info['im_dtype']
cam = info['cam']

In [ ]:
f_raw = np.memmap(join(folder, filename), shape=shape, dtype=dtype, mode='r')
print(np.info(f_raw))

Use the Right Hand Rule to determine the beam coordinates. [Insert image here]. (NEED TO FIX DIPOLE TRANSFER MATRIX FOR VS34; CURRENT CALCULATION IS WRONG AND IS CAUSING SIGN ERROR.
* Cam06 
    * x_slit (x1, x2) = x_beam
    * y_slit (y1) = -y_beam
    * y_screen (y3) = -y_beam
    * x_screen (x3) = -x_beam    
* Cam34
    * x_slit (x1, x2) = -x_beam (Are you sure??? Seems to give the wrong answer.)
    * y_slit (y1) = -y_beam
    * y_screen (y3) = -y_beam
    * x_screen (x3) = +x_beam

In [ ]:
# if cam.lower() == 'cam06':
#     f_raw = f_raw[:, :, ::-1, ::-1, ::-1]
# elif cam.lower() == 'cam34':
#     # a5d = a5d[::-1, ::-1, ::-1, ::-1, :]
#     f_raw = f_raw[:, :, ::-1, ::-1, :]

## Load slit coordinates

In [ ]:
coords_3d = np.load(join(folder, coordfilename))  # [X1, X2, Y1]
coords_3d.shape

In [ ]:
dims = ["x1", "x2", "y1", "y3", "x3"]
dim_to_int = {dim: i for i, dim in enumerate(dims)}

In [ ]:
fig, axes = pplt.subplots(nrows=3, ncols=3, figwidth=6, spanx=False, spany=False)
for i in range(3):
    for j in range(3):
        U = coords_3d[j]
        V = coords_3d[i]
        ax = axes[i, j]
        ax.scatter(U.ravel(), V.ravel(), s=1, color='black')
        ax.axvline(np.mean(U), color='red', alpha=0.15)
        ax.axhline(np.mean(V), color='red', alpha=0.15)
    axes[i, 0].format(ylabel=dims[i])
    axes[-1, i].format(xlabel=dims[i])
plt.savefig('_output/grid3d.png')
plt.show()

Copy the grids into the five-dimensional space.

In [ ]:
X1, X2, Y1 = coords_3d
X1 = utils.copy_into_new_dim(X1, shape[3:], axis=-1)
X2 = utils.copy_into_new_dim(X2, shape[3:], axis=-1)
Y1 = utils.copy_into_new_dim(Y1, shape[3:], axis=-1)

In [ ]:
print('X1.shape =', X1.shape)
print('X2.shape =', X2.shape)
print('Y1.shape =', Y1.shape)

In [ ]:
Y3, X3 = np.meshgrid(np.arange(shape[3]), np.arange(shape[4]), indexing='ij')
Y3 = utils.copy_into_new_dim(Y3, shape[:3], axis=0)
X3 = utils.copy_into_new_dim(X3, shape[:3], axis=0)

In [ ]:
print('Y3.shape =', Y3.shape)
print('X3.shape =', X3.shape)

Make lists of centered coordinates `coords_`.

In [ ]:
X1 = X1 - np.mean(X1)
X2 = X2 - np.mean(X2)
Y1 = Y1 - np.mean(Y1)
Y3 = Y3 - np.mean(Y3)
X3 = X3 - np.mean(X3)
coords_ = [X1, X2, Y1, Y3, X3]

In [ ]:
for i, dim in enumerate(dims):
    print('dim =', dim)
    U = coords_[i]
    axes = [k for k in range(U.ndim) if k != i]
    idx = utils.make_slice(U.ndim, axes, ind=[0, 0, 0, 0])
    print(U[idx])
    print()

## View 5D array in normalized slit-screen coordinates

Correlation between planes are removed... need to be careful interpreting these plots.

In [ ]:
f_raw_min = np.min(f_raw)
f_raw_max = np.max(f_raw)
if f_raw_min < 0:
    print(f'min(f_raw) = {f_raw_min}. Clipping to 0.')
    f_raw = np.clip(f_raw, 0, None)
f_raw = f_raw / f_raw_max

### Interactive 

In [ ]:
mplt.interactive_proj2d(f_raw / f_raw_max, dims=['x1', 'x2', 'y1', 'y3', 'x3'], 
                        slider_type='int', default_ind=(4, 3))

In [ ]:
mplt.interactive_proj2d(f_raw / f_raw_max, dims=['x1', 'x2', 'y1', 'y3', 'x3'], 
                        slider_type='range', default_ind=(4, 3))

### Projections 

In [ ]:
for norm in [None, 'log']:
    axes = mplt.corner(
        f_raw,
        labels=dims,
        norm=norm,
        diag_kind='None',
        fig_kws=dict(),
        prof=True,
        prof_kws=dict(lw=1.0, alpha=0.5, scale=0.12),
        handle_log='floor',
    )
    plt.savefig(f"_output/slitscreen_corner_log{norm=='log'}.png")
    plt.show()

### Slices

Compute the indices of the maximum pixel in the 5D array.

In [ ]:
ind_max = np.unravel_index(np.argmax(f_raw), f_raw.shape)
print(ind_max)

In [ ]:
axes_slice = [(k, j, i) for i in range(f_raw.ndim) for j in range(i) for k in range(j)]
axes_view = [tuple([i for i in range(f_raw.ndim) if i not in axis])
             for axis in axes_slice]
for axis, axis_view in zip(axes_slice, axes_view):
    idx = utils.make_slice(5, axis, [ind_max[i] for i in axis])
    f_raw_slice = f_raw[idx]
    f_raw_slice = f_raw_slice / np.max(f_raw_slice)

    dim1, dim2 = [dims[i] for i in axis_view]
    
    fig, plot_axes = pplt.subplots(ncols=2)
    for ax, norm in zip(plot_axes, [None, 'log']):
        mplt.plot_image(f_raw_slice, ax=ax, norm=norm, colorbar=True, handle_log='floor')
    plot_axes.format(xlabel=dim1, ylabel=dim2)
    string = '_output/slitscreen_slice_'
    for i in axis:
        string += f'_{dims[i]}-{ind_max[i]}'
    plt.savefig(string + '.png')
    plt.show()

In [ ]:
utils.save_pickle('_output/info.pkl', info)
file = open('_output/info.txt', 'w')
for key, value in info.items():
    file.write(f'{key}: {value}\n')
file.close()